In [1]:
import tkinter as tk
import pandas as pd
import pyrsm as rsm

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import nltk
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    

In [26]:
    
interaction = pd.read_csv('data/RAW_interactions.csv')
recipe = pd.read_csv("data/RAW_recipes.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'RAW_interactions.csv'

In [27]:

# drop rows with na
interaction = interaction.dropna()
# reset_index since we dropped some rows
interaction = interaction.reset_index()
# drop unneeded col
interaction = interaction.drop(columns=["index"])
# take first 100 rows for example
interaction_tmp = interaction.loc[0:99, :]
# groupby recipe_id
interaction_groupby = interaction.groupby(by=["recipe_id"])["review"].count().sort_values(ascending=False)
# transform groupbyed result to dataframe
tmp = interaction_groupby.to_frame()
tmp = tmp.rename(columns={"review": "review_count"})
# see how many recipes are there
len(tmp.query("review_count > 80")) # 1067
# if > 80, assgin 1 in the selected col
tmp["selected"] = rsm.ifelse(tmp.review_count > 80, 1, 0)
# reset_index
tmp = tmp.reset_index()
# merge
interaction_merge = interaction.merge(tmp, on="recipe_id", how="left") # left_join
# get selected data only
interaction_selected = interaction_merge.query("selected == 1")
# reset_index
interaction_selected = interaction_selected.reset_index()
# drop unneeded col
interaction_selected = interaction_selected.drop(columns=["index"])
# model to preprocessing reviews

nltk.download('punkt') # downloads you a model                                                                                                         
nltk.download('stopwords') 
ps = PorterStemmer() 
# return a list of tokens
def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences
    sentences = sent_tokenize(doc)
    # step 2: get tokens
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        # step 3 (optional): stemming
        if stemming:
            words = [ps.stem(word) for word in words]
        if need_sent:
            tokens.append(words)
        else:
            tokens += words
    return [w.lower() for w in tokens if w.isalpha()]
review_split_100 = interaction_selected.iloc[:100].review.apply(pre_processing_by_nltk)
#read two txt to recognize pos or neg words
pos_words = open("positive-words.txt").read().split('\n')
neg_words = open("negative-words.txt").read().split('\n')
#function to calculate percentages
def percentage_PN(re): 
    #calculate positive word
    numPosWords = 0
    for word in re:
        if word in pos_words:
            numPosWords += 1
    
    # calculate negative words    
    numNegWords = 0   
    for word in re:
        if word in neg_words:
            numNegWords += 1
    
    #decide if positiv eor negative
    if numPosWords > numNegWords:
        return("Positive")
    elif numNegWords > numPosWords:
        return("Negative")
    elif numNegWords == numPosWords:
        return("Neutral") 
# apply on whole data
per_results = review_split_100.apply(percentage_PN)
#combine percentage results with recipe id
d = {'recipe_id':interaction.recipe_id ,'per_results':per_results}
df_per = pd.DataFrame(data = d)
df_per = df_per.dropna()
# combine num_df_per with recipe to get final dataset
df_per = df_per.rename(columns={"recipe_id": "id"})
# final dataset
DF = df_per.merge(recipe, on="id", how="left") # left_join
# specify X and y
X, y = DF.loc[:, ["minutes", "n_steps", "n_ingredients"]], DF["per_results"]
# splitting the train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=False)
# final model
clf_cv = MLPClassifier(
    activation="tanh",
    solver="lbfgs",
    alpha=0.01,
    hidden_layer_sizes=(4,4),
    random_state=1234,
    max_iter=10000,
).fit(X_train, y_train)

[nltk_data] Error loading punkt: <urlopen error [WinError 10061]
[nltk_data]     由于目标计算机积极拒绝，无法连接。>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10061]
[nltk_data]     由于目标计算机积极拒绝，无法连接。>


In [35]:
# collect x
tmp = X_train.copy()
tmp = tmp.loc[0:0, :]

import tkinter as tk

# 第1步，实例化object，建立窗口window
window = tk.Tk()
 
# 第2步，给窗口的可视化起名字
window.title('Recipe Sentiment Prediction')
 
# 第3步，设定窗口的大小(长 * 宽)
window.geometry('600x600')  # 这里的乘是小x

# 第4步，加载 wellcome image
canvas = tk.Canvas(window, width=500, height=380, bg='yellow')
image_file = tk.PhotoImage(file='picture.png')
image = canvas.create_image(230, 0, anchor='n', image=image_file)
canvas.pack(side='top')
tk.Label(window, text='Recipe Review Prediction',font=('Arial', 20)).pack()


# 第5步，在图形界面上设定输入框控件entry并放置控件
e1 = tk.Entry(window, show=None, font=('Arial', 15))  # 显示成明文形式

e2 = tk.Entry(window, show=None, font=('Arial', 15))  # 显示成明文形式

e3 = tk.Entry(window, show=None, font=('Arial', 15))  # 显示成明文形式


# def 
def to_predict():
    a = e1.get()
    a = int(a)
    
    b = e2.get()
    b = int(b)
    
    c = e3.get()
    c = int(c)
    
    #input variables
    tmp.loc[0, "minutes"] = a
    tmp.loc[0, "n_steps"] = b
    tmp.loc[0, "n_ingredients"] = c
    
    result = clf_cv.predict(tmp)
    
    t.insert('insert', result)
    
tk.Label(window, text='minutes:', font=('Arial', 12)).place(x=10, y=430)    
tk.Label(window, text='n_steps:', font=('Arial', 12)).place(x=10, y=460)  
tk.Label(window, text='n_ingredients:', font=('Arial', 12)).place(x=10, y=490)  

button = tk.Button(window, command = to_predict, text = "predict")

# pack
e1.pack()
e2.pack()
e3.pack()

button.pack()

t = tk.Text(window, height=3)
t.pack()

window.mainloop()